In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]

feats_files = [
# '../features/glove_cnn2d_4_feat.pkl',
'../features/glove_cnn_gru_4_feat.pkl',
# '../features/glove_cnn_v1_4_feat.pkl',
'../features/glove_cnn_v2_4_feat.pkl',
# '../features/glove_cudnn_gru_4_feat.pkl',
# '../features/glove_gru_v1_4_feat.pkl',
# '../features/glove_lstm_v1_4_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_attention_fasttext_10_feat.pkl',
#'../features/lstm_attention_fasttext_4_feat.pkl',
'../features/lstm_attention_glove_4_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/muse_cnn2d_4_feat.pkl',
'../features/muse_cnn_gru_4_feat.pkl',
# '../features/muse_cnn_v1_4_feat.pkl',
# '../features/muse_cnn_v2_4_feat.pkl',
# '../features/muse_cudnn_gru_4_feat.pkl',
# '../features/muse_gru_v1_4_feat.pkl',
# '../features/muse_lstm_v1_4_feat.pkl',
'../features/no_pretrained_cnn2d_4_feat.pkl',
'../features/no_pretrained_cnn_gru_4_feat.pkl',
'../features/no_pretrained_cnn_v1_4_feat.pkl',
'../features/no_pretrained_cnn_v2_4_feat.pkl',
'../features/no_pretrained_cudnn_gru_4_feat.pkl',
'../features/no_pretrained_gru_v1_4_feat.pkl',
'../features/no_pretrained_lstm_v1_4_feat.pkl',
'../features/other_feat.pkl',
#'../features/pool_gru_fasttext_4_feat.pkl',
'../features/pool_gru_fasttext_adj1_10_feat.pkl',
'../features/pool_gru_glove_4_feat.pkl',
'../features/tfidf_feat1.pkl',
'../features/tfidf_feat2.pkl',
]

for feat in feats_files:
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/glove_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_attention_fasttext_10_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_attention_glove_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/muse_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/muse_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/no_pretrained_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/no_pretrained_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/no_pretrained_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/no_pretrained_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path .

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss'):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        d_test = test_x
        
        # share params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': 3,
                #'scale_pos_weight':0.9,
                'metric': met, # or auc
                'data_random_seed': 2,
                'learning_rate':lr,
#                 'bagging_fraction': bagging_fraction,
#                 'bagging_freq':bag_frec,
                'feature_fraction': feature_fraction,
            
                }
        if met == 'auc':
            s_round = 100
        else:
            s_round = 50
        # train for each class
        for i in range(6):
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(i)
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        train_auc_l = train_auc_l/6
        val_auc_l = val_auc_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        print('this fold auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        all_train_auc_l += train_auc_l/k
        all_val_auc_l += val_auc_l/k
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

done


In [3]:
%%time
lgb_res = simple_ens('lgb',10,233,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_some_fold10.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# select feats
# all loss avg 0.0323893274343 0.0361490549659
# all auc avg 0.994298774643 0.991023792329

0
ls 0.0671443296254 0.0759746581004 auc 0.991462866032 0.988517303567
1
ls 0.0181591028857 0.0207497631293 auc 0.99404007815 0.991996481366
2
ls 0.0371952483473 0.0409123574943 auc 0.995749616368 0.993835139285
3
ls 0.00534578047567 0.0072626488765 auc 0.99811532163 0.997390917956
4
ls 0.0502265227801 0.0566556363124 auc 0.991605365266 0.987752420139
5
ls 0.0150275963358 0.0194000484431 auc 0.995241328373 0.988993138991
this fold avg train 0.0321830967417 avg val 0.0368258520593
this fold auc train 0.99436909597 auc val 0.991414233551
0
ls 0.0679368783245 0.0776695665038 auc 0.991309050248 0.98653064089
1
ls 0.0178200476884 0.019134640461 auc 0.994370666135 0.993633884859
2
ls 0.0357987446699 0.0359203285262 auc 0.996120192159 0.995775949697
3
ls 0.0051741439062 0.00727631422216 auc 0.998361766923 0.993043137972
4
ls 0.0483293958499 0.0492295038218 auc 0.992388213918 0.991512862059
5
ls 0.0162447011785 0.0167722700692 auc 0.994231286295 0.986353686515
this fold avg train 0.03188398526